<a href="https://colab.research.google.com/github/Radwan1244/Dog-breeding-classifier-/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import sys
import logging
import argparse
from typing import Dict, List, Any, Set, Tuple
from dataclasses import dataclass, asdict

# ML and Image Processing Imports
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms

class ImagePreprocessor:
    """
    Image preprocessing utility for neural network input preparation.
    """
    def __init__(self):
        """
        Initialize image preprocessing transformations.
        """
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def prepare(self, image_path: str) -> torch.Tensor:
        """
        Preprocess image for neural network input.

        Args:
            image_path: Path to input image

        Returns:
            Preprocessed image tensor
        """
        try:
            image = Image.open(image_path).convert('RGB')
            return self.transform(image).unsqueeze(0)
        except Exception as e:
            raise RuntimeError(f"Image preprocessing failed: {e}")

class DogBreedClassifier:
    """
    CNN-based dog breed classification model.
    """
    def __init__(self, model_arch: str = 'vgg'):
        """
        Initialize pre-trained CNN model for classification.

        Args:
            model_arch: Selected model architecture
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self._load_model(model_arch)
        self._load_class_labels()

    def _load_model(self, model_arch: str):
        """
        Load pre-trained CNN model based on architecture.

        Args:
            model_arch: Selected model architecture
        """
        try:
            if model_arch == 'vgg':
                self.model = models.vgg16(pretrained=True)
            elif model_arch == 'resnet':
                self.model = models.resnet50(pretrained=True)
            elif model_arch == 'alexnet':
                self.model = models.alexnet(pretrained=True)
            else:
                raise ValueError(f"Unsupported model architecture: {model_arch}")

            self.model.eval()
            self.model.to(self.device)
        except Exception as e:
            raise RuntimeError(f"Model loading failed: {e}")

    def _load_class_labels(self):
        """
        Load ImageNet class labels for decoding predictions.
        """
        from torchvision.models import VGG16_Weights
        self.class_labels = VGG16_Weights.DEFAULT.meta['categories']

    def predict(self, image_tensor: torch.Tensor) -> str:
        """
        Predict dog breed from input image.

        Args:
            image_tensor: Preprocessed image tensor

        Returns:
            Predicted dog breed label
        """
        try:
            with torch.no_grad():
                image_tensor = image_tensor.to(self.device)
                outputs = self.model(image_tensor)
                _, predicted = torch.max(outputs, 1)
                breed_index = predicted.item()

                return self.class_labels[breed_index]
        except Exception as e:
            raise RuntimeError(f"Prediction failed: {e}")

class ModelEvaluator:
    """
    Utility for computing advanced model evaluation metrics.
    """
    @staticmethod
    def compute_precision(tp: int, fp: int) -> float:
        """
        Calculate precision metric.

        Args:
            tp: True positives
            fp: False positives

        Returns:
            Precision score
        """
        return tp / (tp + fp) if (tp + fp) > 0 else 0.0

    @staticmethod
    def compute_recall(tp: int, fn: int) -> float:
        """
        Calculate recall metric.

        Args:
            tp: True positives
            fn: False negatives

        Returns:
            Recall score
        """
        return tp / (tp + fn) if (tp + fn) > 0 else 0.0

    @staticmethod
    def compute_f1_score(precision: float, recall: float) -> float:
        """
        Calculate F1 score.

        Args:
            precision: Precision metric
            recall: Recall metric

        Returns:
            F1 score
        """
        return (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
